In [1]:
import os

In [2]:
%pwd

'/Users/rociocuatecontzi/Desktop/programming/electricity-forecast/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rociocuatecontzi/Desktop/programming/electricity-forecast'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/gorogandras/electricity-forecast.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="gorogandras"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d28d223a6b24642b9af69a8bb4f5c10d96764125"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    train_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    columns: str
    mlflow_uri: str

In [7]:
from eForecaster.constants import *
from eForecaster.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGRBoost
        schema =  self.schema

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            train_data_path=config.train_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.TARGET_COLUMN.name,
            columns = schema.COLUMNS,
            mlflow_uri="https://dagshub.com/gorogandras/electricity-forecast.mlflow"
           
        )
            

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import seaborn as sns
from matplotlib import pyplot as plt

In [10]:

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        test_x = test_data[[item for item in self.config.columns.keys()]]
        test_y = test_data[[self.config.target_column]]
        
        print(test_x)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="XGRBoostModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    
    def get_scatterplot(self):
        test_data = pd.read_csv(self.config.test_data_path)
        train_data = pd.read_csv(self.config.train_data_path)
        model = joblib.load(self.config.model_path)
        test_data = test_data[["datetime", "power"]]
        train_data = train_data[["datetime", "power"]]
        start_date = test_data["datetime"].iloc[[0]].values[0]

        test_data["datetime"] = pd.to_datetime(test_data["datetime"], 
                                               format="%Y-%m-%d %H:%M:%S")
        train_data["datetime"] = pd.to_datetime(train_data["datetime"], 
                                               format="%Y-%m-%d %H:%M:%S")
        #fig, ax = plt.subplots(figsize=(15,5))
        #train_data.plot(ax=ax, label='Training Set', title="Data train/Test Split")
        #test_data.plot(ax=ax, label='Test Set')
        #ax.axvline(start_date, color='black', ls='--')
        #ax.legend(['Training Set', 'Test set'])
        test_data['prediction'] = model.predict(test_data['datetime'])
        ax = train_data[['power']].plot(figsize=(15,5))
        plt.show()
        plt.clf() 
        print(test_data)



In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    #model_evaluation_config.get_scatterplot()
except Exception as e:
    raise e

[2024-02-01 19:18:35,343: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-01 19:18:35,347: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-01 19:18:35,350: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-01 19:18:35,355: INFO: common: created directory at: artifacts]
[2024-02-01 19:18:35,357: INFO: common: created directory at: artifacts/model_evaluation]


In [20]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
    model_evaluation_config.get_scatterplot()
except Exception as e:
    raise e

[2024-01-30 22:14:04,722: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-30 22:14:04,726: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-30 22:14:04,728: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-30 22:14:04,730: INFO: common: created directory at: artifacts]
[2024-01-30 22:14:04,732: INFO: common: created directory at: artifacts/model_evaluation]
       hour  dayofweek  quarter  month  year  dayofyear  minute
0         0          0        4     10  2021        277       0
1         0          0        4     10  2021        277      15
2         0          0        4     10  2021        277      30
3         0          0        4     10  2021        277      45
4         1          0        4     10  2021        277       0
...     ...        ...      ...    ...   ...        ...     ...
34940    23          6        4     10  2022        275       0
34941    23          6        4     10  2022        275      15
34

/Users/rociocuatecontzi/anaconda3/envs/eforecast/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'XGRBoostModel' already exists. Creating a new version of this model...
2024/01/30 22:14:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: XGRBoostModel, version 7
Created version '7' of model 'XGRBoostModel'.
